In [1]:
import pandas as pd

C:\Users\l3n3c\AppData\Local\Temp\ipykernel_28428\4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
agri_gdp_data = pd.read_csv('./agri_gdp_data.csv')
agri_gdp_data = agri_gdp_data.drop(columns='Unnamed: 0')


Index(['Area', 'Year', 'Savanna fires', 'Forest fires', 'Crop Residues',
       'Rice Cultivation', 'Drained organic soils (CO2)',
       'Pesticides Manufacturing', 'Food Transport', 'Forestland',
       'Net Forest conversion', 'Food Household Consumption', 'Food Retail',
       'On-farm Electricity Use', 'Food Packaging',
       'Agrifood Systems Waste Disposal', 'Food Processing',
       'Fertilizers Manufacturing', 'IPPU', 'Manure applied to Soils',
       'Manure left on Pasture', 'Manure Management', 'Fires in organic soils',
       'Fires in humid tropical forests', 'On-farm energy use',
       'Rural population', 'Urban population', 'Total Population - Male',
       'Total Population - Female', 'total_emission', 'Average Temperature °C',
       'Code', 'gdp'],
      dtype='object')

Train test split

In [8]:
# imports
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [12]:
agri_gdp_data.columns

Index(['Area', 'Year', 'Savanna fires', 'Forest fires', 'Crop Residues',
       'Rice Cultivation', 'Drained organic soils (CO2)',
       'Pesticides Manufacturing', 'Food Transport', 'Forestland',
       'Net Forest conversion', 'Food Household Consumption', 'Food Retail',
       'On-farm Electricity Use', 'Food Packaging',
       'Agrifood Systems Waste Disposal', 'Food Processing',
       'Fertilizers Manufacturing', 'IPPU', 'Manure applied to Soils',
       'Manure left on Pasture', 'Manure Management', 'Fires in organic soils',
       'Fires in humid tropical forests', 'On-farm energy use',
       'Rural population', 'Urban population', 'Total Population - Male',
       'Total Population - Female', 'total_emission', 'Average Temperature °C',
       'Code', 'gdp'],
      dtype='object')

In [13]:
agri_gdp_tr = agri_gdp_data.copy()
agri_gdp_tr.head()
agri_gdp_data = agri_gdp_tr.drop("total_emission", axis=1)
agri_gdp_labels = agri_gdp_tr["total_emission"].copy()

X_train, X_test, y_train, y_test = train_test_split(agri_gdp_data, agri_gdp_labels, test_size=0.2)

pipeline

In [15]:
imputer = SimpleImputer(strategy="most_frequent")
std_scaler = StandardScaler()

num_pipeline = Pipeline([
    ("imputer", imputer),
    ("std_scaler", std_scaler)
])
num_col = agri_gdp_data.select_dtypes(['number']).columns

# converting categorical data
cat_encoder = OneHotEncoder(handle_unknown="ignore")
cat_col = agri_gdp_data.select_dtypes(['object']).columns

# full pipeline
full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_col),
    ("cat", cat_encoder, cat_col)
])

In [ ]:
training_prepared = full_pipeline.fit_transform(X_train)
test_prepared = full_pipeline.fit(X_test) # do we put this through the pipeline i forget

regression & error

In [20]:
import numpy as np
from sklearn.metrics import mean_squared_error

def calculate_mse_rmse(predictions, labels):
    mse = mean_squared_error(labels, predictions)
    rmse = np.sqrt(mse)
    return mse, rmse

In [25]:
# Linear Regression

from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(training_prepared, y_train)

lin_pred = lin_reg.predict(training_prepared)

lin_mse, lin_rmse = calculate_mse_rmse(lin_pred, y_train)
print("training mse: ", lin_mse)
print("training rmse: ", lin_rmse)

lin_test_pred = lin_reg.predict(test_prepared)

lin_mse, lin_rmse = calculate_mse_rmse(lin_test_pred, y_test)
print("test mse: ", lin_mse)
print("test rmse: ", lin_rmse)

training mse:  19.440951826662193
training rmse:  4.409189475023974


ValueError: Expected 2D array, got scalar array instead:
array=ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('std_scaler',
                                                  StandardScaler())]),
                                 Index(['Year', 'Savanna fires', 'Forest fires', 'Crop Residues',
       'Rice Cultivation', 'Drained organic soils (CO2)',
       'Pesticides Manufacturing', 'Food Transport', 'Forestland',
       'Net Forest conversion', 'Food Household Consumption', 'F...
       'Fertilizers Manufacturing', 'IPPU', 'Manure applied to Soils',
       'Manure left on Pasture', 'Manure Management', 'Fires in organic soils',
       'Fires in humid tropical forests', 'On-farm energy use',
       'Rural population', 'Urban population', 'Total Population - Male',
       'Total Population - Fe

tuning hyperparameters?

evaluation & plots